### assess scores using mubind

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mubind
import pandas as pd
import glob
import os
import anndata


In [3]:
cd ~/workspace/theislab/mubind-pipeline/notebooks/pipeline

/mnt/c/Users/IgnacioIbarra/Dropbox/workspace/theislab/mubind-pipeline/notebooks/pipeline


In [4]:
import utils

In [ ]:
import os
import datetime
def old_file(path, n_hours):
    assert os.path.exists(path)
    mod_time = os.path.getctime(path)
    return datetime.fromtimestamp(mod_time) > datetime.now() - datetime.timedelta(hours=n_hours)

In [5]:
import torch
import mubind as mb
import numpy as np
import pickle
import datetime
rootdir = '/mnt/f/workspace/theislab/mubind/data/*/*/mubind_output/obs*/*/*'
overwrite = True

df = []
for p in glob.glob(rootdir):

    # print(p)
    # if not '/mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/episcanpy/mubind_output/obs100/pwms/' in p:
    #     continue
    # continue

    print(p)
    if not 'model_0.pt' in p:
        continue

    print('loading model')
        
    runtime_path = p.replace('model_0.pt', 'running_time.pkl')
    time_diff = pickle.load(open(runtime_path, 'rb')) if os.path.exists(runtime_path) else None

    print(time_diff, type(time_diff))
    if not isinstance(time_diff, datetime.timedelta):
        print('time log not ok....skip')
        continue
        
    scores_path = p.replace('.pt', '_scores.csv')
    print(os.path.exists(scores_path), scores_path)
    if os.path.exists(scores_path) and not overwrite:
        df2 = pd.read_csv(scores_path, index_col=0)
        df2['n_obs'] = int(scores_path.split('/')[-3].replace('obs', ''))
        df2['feat_selection'] = scores_path.split('/')[-4].replace('obs', '')
        df.append(df2)
        continue

    # 

    res = []

    model = torch.load(p)

    feature_selection_rule = p.split('/')[-4]

    print(p)

    os.makedirs("results", exist_ok=True)

    n_obs = p.split('/')[-3]
    dataset_name = p.split('/')[-6]

    ad_path = os.path.join(os.path.dirname(p),
                            '../../../scbasset_input/%s/ad.h5ad' % n_obs)
    splits_path = os.path.join(os.path.dirname(p),
                                '../../../scbasset_input/%s/splits.h5' % n_obs)
    peaks_path = os.path.join(os.path.dirname(p),
                                '../../../scbasset_input/%s/peaks.bed' % n_obs)
    # print(os.path.exists(ad_path), ad_path)
    # print(os.path.exists(splits_path), splits_path)
    # print(os.path.exists(peaks_path), peaks_path)

    # read h5ad file
    ad = anndata.read_h5ad(ad_path)

    # print('preparing sequences test...')
    # print(dataset_name)

    species = 'hg38' if (dataset_name != 'noack_2022' and dataset_name != 'pancreatic_endocrinogenesis') else 'mm10'
    gen_path = '/mnt/c/Users/IgnacioIbarra/Dropbox/annotations/%s/genome/%s.fa' % (species, species)

    seqs = pd.read_csv(peaks_path, sep='\t', header=None)
    seqs.columns = ['chr', 'start', 'end']

    assert seqs.shape[0] == ad.shape[1]
    # print(seqs.shape, ad.shape)

    # fix seq coordinates
    ad.var['chr'] = np.where(ad.var_names.str.contains(':'), ad.var_names.str.split(':').str[0], ad.var_names.str.split('-').str[0])
    ad.var['chr'] = np.where(~ad.var['chr'].str.contains('chr'), 'chr', '') + ad.var['chr']
    ad.var['start'] = [s.split(':')[1].split('-')[0] if ':' in s else s.split('-')[1] for s in ad.var_names]
    # ad.var['start'] = np.where(~ad.var_names.str.contains(':'), ad.var_names.str.split('-').str[1], ad.var_names.str.split(':').str[1].str.split('-').str[0])
    ad.var['end'] = np.where(~ad.var_names.str.contains(':'), ad.var_names.str.split('-').str[2], ad.var_names.str.split('-').str[1])

    seqs = ad.var[['chr', 'start', 'end']]

    all_seqs = mb.bindome.tl.get_sequences_from_bed(seqs[['chr', 'start', 'end']],
                                                    gen_path=gen_path,
                                                    uppercase=True)
                                                # gen_path='../../../annotations/dm6/genome/dm6.fa')
    keys_test = set([s[0] for s in all_seqs])
    # ad_test = ad_test[:,ad_test.var['k_summit'].isin(keys_test)]

    # convert to csr matrix
    import h5py
    with h5py.File(splits_path, 'r') as hf:
        train_ids = hf['train_ids'][:]
        val_ids = hf['val_ids'][:]
        test_ids = hf['test_ids'][:]


    assert ad.shape[1] == len(all_seqs)
    # print(ad_train.shape, ad_test.shape)


    import numpy as np
    import torch.utils.data as tdata

    res = []

    ad_train = ad[:,train_ids]
    ad_val = ad[:,val_ids]
    ad_test = ad[:,test_ids]
    ad_train.shape, ad_val.shape, ad_test.shape


    all_seqs = np.array(all_seqs)
    train_seqs, val_seqs, test_seqs = all_seqs[train_ids], all_seqs[val_ids], all_seqs[test_ids]

    for next_ad, next_seqs, next_label in zip([ad_train, ad_val, ad_test],
                                            [train_seqs, val_seqs, test_seqs],
                                            ['train', 'val', 'test']):
        next_df = utils.prepare_df(next_seqs, next_ad)

        dataset = mb.datasets.SelexDataset(next_df,
                                                n_rounds=next_df.shape[1],
                                                enr_series=False)
        dataloader = tdata.DataLoader(dataset=dataset,
                                batch_size=512,
                                shuffle=True)

        roc_auc = utils.get_auroc(model, dataloader)
        pr_auc = utils.get_auprc(model, dataloader)
        scores = mb.tl.scores(model, dataloader)

        print('here...')
        res.append([dataset_name, next_label, feature_selection_rule, 'roc_auc', roc_auc, time_diff.total_seconds(), p])
        res.append([dataset_name, next_label, feature_selection_rule, 'pr_auc', pr_auc, time_diff.total_seconds(), p])
        res.append([dataset_name, next_label, feature_selection_rule, 'r2', scores['r2_counts'], time_diff.total_seconds(), p])

    res = pd.DataFrame(res, columns=['dataset', 'group', 'feat_selection', 'metric', 'value', 'time', 'model_path'])

    res['n_obs'] = int(scores_path.split('/')[-3].replace('obs', ''))
    res.to_csv(scores_path)
    
    df.append(res)

res = pd.concat(df)
res['model'] = 'mubind'
print('')

/mnt/f/workspace/theislab/mubind/data/noack_2022/random/mubind_output/obs500/denovo/model.csv
/mnt/f/workspace/theislab/mubind/data/noack_2022/random/mubind_output/obs500/denovo/model_0.pt
loading model
0:08:03.511847 <class 'datetime.timedelta'>
True /mnt/f/workspace/theislab/mubind/data/noack_2022/random/mubind_output/obs500/denovo/model_0_scores.csv
/mnt/f/workspace/theislab/mubind/data/noack_2022/random/mubind_output/obs500/denovo/model_0.pt
/tmp/tmpc66zq4bn
genome hg19 False
/mnt/c/Users/IgnacioIbarra/Dropbox/annotations/mm10/genome/mm10.fa
True /mnt/c/Users/IgnacioIbarra/Dropbox/annotations/mm10/genome/mm10.fa
running bedtools...
['bedtools', 'getfasta', '-fi', '/mnt/c/Users/IgnacioIbarra/Dropbox/annotations/mm10/genome/mm10.fa', '-bed', '/tmp/tmpc66zq4bn', '-fo', '/tmp/tmph2kspzj2']
bedtools getfasta -fi /mnt/c/Users/IgnacioIbarra/Dropbox/annotations/mm10/genome/mm10.fa -bed /tmp/tmpc66zq4bn -fo /tmp/tmph2kspzj2
(1351, 501)
# cells 500
# peaks 1351
selected (1342, 500)
here...
(

In [6]:
res.to_csv('results_mubind.csv')